In [26]:
import numpy as np
import torch
import torch.nn.functional as  F
import trimesh
import pytorch3d
from pytorch3d.utils import ico_sphere

from src.config import get_parser
from src.util import make_faces


torch.set_printoptions(sci_mode=False, precision=4)

In [2]:
config = get_parser().parse_args(args=[])

# Set the cuda device
device = torch.device("cuda:0")


blueprint =  np.load(config.blueprint)        
points = torch.tensor(blueprint['points'])[0]
normals = torch.tensor(blueprint['normals'])[0]
points.shape, normals.shape

(torch.Size([3, 256, 256]), torch.Size([3, 256, 256]))

In [3]:
# Vertex normals are calculated by averaging all of the 
# face normals of triangles of which the vertex is a part. 
_, w, h = points.shape
faces = make_faces(w, h)
faces

array([[    0,     1,   256],
       [    1,   257,   256],
       [    1,     2,   257],
       ...,
       [65278, 65534, 65533],
       [65278, 65279, 65534],
       [65279, 65535, 65534]])

In [4]:
vertices = np.array([[ 0.82667452,  0.89591247,  0.91638623],
                        [ 0.10045271,  0.50575086,  0.73920507],
                        [ 0.06341482,  0.17413744,  0.6316301 ],
                        [ 0.75613029,  0.82585983,  0.10012549],
                        [ 0.45498342,  0.5636221 ,  0.10940527],
                        [ 0.46079863,  0.54088544,  0.1519899 ],
                        [ 0.61961934,  0.78550213,  0.43406491],
                        [ 0.12654252,  0.7514213 ,  0.18265301],
                        [ 0.94441365,  0.00428673,  0.46893573],
                        [ 0.79083297,  0.70198129,  0.75670947]] )
faces = np.array( [[0,1,2],
                   [0,2,3], 
                   [1,2,3], 
                   [1,4,5], 
                   [2,5,6], 
                   [6,3,7], 
                   [9,8,7]])

def normalize_v3(arr):
    ''' Normalize a numpy array of 3 component vectors shape=(n,3) '''
    lens = np.sqrt( arr[:,0]**2 + arr[:,1]**2 + arr[:,2]**2 )
    arr[:,0] /= lens
    arr[:,1] /= lens
    arr[:,2] /= lens                
    return arr

#Create a zeroed array with the same type and shape as our vertices i.e., per vertex normal
norm = np.zeros( vertices.shape, dtype=vertices.dtype )
#Create an indexed view into the vertex array using the array of three indices for triangles
tris = vertices[faces]
#Calculate the normal for all the triangles, by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle             
n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
# n is now an array of normals per triangle. The length of each normal is dependent the vertices, 
# we need to normalize these, so that our next step weights each normal equally.
normalize_v3(n)
# now we have a normalized array of normals, one per triangle, i.e., per triangle normals.
# But instead of one per triangle (i.e., flat shading), we add to each vertex in that triangle, 
# the triangles' normal. Multiple triangles would then contribute to every vertex, so we need to normalize again afterwards.
# The cool part, we can actually add the normals through an indexed view of our (zeroed) per vertex normal array
norm[ faces[:,0] ] += n
norm[ faces[:,1] ] += n
norm[ faces[:,2] ] += n
normalize_v3(norm)
norm

array([[ 0.68647611, -0.72714585,  0.00307691],
       [-0.41663866, -0.79746992,  0.4364103 ],
       [ 0.561194  , -0.48419887,  0.67127696],
       [ 0.53644249,  0.60476166,  0.58863638],
       [-0.49963522, -0.79064121, -0.35390836],
       [ 0.17280246, -0.97347702, -0.14993929],
       [ 0.84690504,  0.45590931,  0.27367601],
       [ 0.61293173,  0.4128088 , -0.67372367],
       [ 0.61293173,  0.4128088 , -0.67372367],
       [ 0.61293173,  0.4128088 , -0.67372367]])

In [5]:
mesh = trimesh.Trimesh(
    vertices=vertices,
    faces=faces,
    #face_normals=norm
    vertex_normals=norm,
)
mesh.show();

In [6]:
norm.shape, faces.shape, vertices.shape

((10, 3), (7, 3), (10, 3))

In [7]:
src_mesh = ico_sphere(0, device)
ico_vertices = src_mesh.verts_list()[0]
ico_vertex_normals = src_mesh.verts_normals_list()[0]
ico_faces = src_mesh.faces_list()[0]
ico_face_normals = src_mesh.faces_normals_list()[0]
ico_vertices.shape, ico_vertex_normals.shape, ico_faces.shape, ico_face_normals.shape

(torch.Size([12, 3]),
 torch.Size([12, 3]),
 torch.Size([20, 3]),
 torch.Size([20, 3]))

In [8]:
tris = ico_vertices[ico_faces]
tris.shape

torch.Size([20, 3, 3])

In [9]:
# torch.cross(vertices_faces[:, 2] - vertices_faces[:, 1],
#             vertices_faces[:, 0] - vertices_faces[:, 1],
#                     dim=1)

vec1 = ico_vertices[ico_faces[:, 1]] - ico_vertices[ico_faces[:, 0]]
vec2 = ico_vertices[ico_faces[:, 2]] - ico_vertices[ico_faces[:, 0]]
face_norm = F.normalize(vec1.cross(vec2), p=2, dim=-1)  # [F, 3]
face_norm.shape

torch.Size([20, 3])

In [10]:
torch.allclose(face_norm, ico_face_normals)

True

In [ ]:
faces_packed = self.faces_packed()
verts_packed = self.verts_packed()
verts_normals = torch.zeros_like(verts_packed)
vertices_faces = verts_packed[faces_packed]

# NOTE: this is already applying the area weighting as the magnitude
# of the cross product is 2 x area of the triangle.
# pyre-fixme[16]: `Tensor` has no attribute `index_add`.
verts_normals = verts_normals.index_add(
    0,
    faces_packed[:, 1],
    torch.cross(
        vertices_faces[:, 2] - vertices_faces[:, 1],
        vertices_faces[:, 0] - vertices_faces[:, 1],
        dim=1,
    ),
)
verts_normals = verts_normals.index_add(
    0,
    faces_packed[:, 2],
    torch.cross(
        vertices_faces[:, 0] - vertices_faces[:, 2],
        vertices_faces[:, 1] - vertices_faces[:, 2],
        dim=1,
    ),
)
verts_normals = verts_normals.index_add(
    0,
    faces_packed[:, 0],
    torch.cross(
        vertices_faces[:, 1] - vertices_faces[:, 0],
        vertices_faces[:, 2] - vertices_faces[:, 0],
        dim=1,
    ),
)

self._verts_normals_packed = torch.nn.functional.normalize(
    verts_normals, eps=1e-6, dim=1
)

In [11]:
ico_faces, ico_faces.max()

(tensor([[ 0, 11,  5],
         [ 0,  5,  1],
         [ 0,  1,  7],
         [ 0,  7, 10],
         [ 0, 10, 11],
         [ 1,  5,  9],
         [ 5, 11,  4],
         [11, 10,  2],
         [10,  7,  6],
         [ 7,  1,  8],
         [ 3,  9,  4],
         [ 3,  4,  2],
         [ 3,  2,  6],
         [ 3,  6,  8],
         [ 3,  8,  9],
         [ 4,  9,  5],
         [ 2,  4, 11],
         [ 6,  2, 10],
         [ 8,  6,  7],
         [ 9,  8,  1]], device='cuda:0'),
 tensor(11, device='cuda:0'))

In [18]:
def vertex_tris(faces):
    res = []
    for vid in range(faces.max()+1):
        vertex_faces = []
        for fid, face in enumerate(faces):
            if vid in face:
                vertex_faces.append(fid)
        res.append(vertex_faces)
    return res

vts = vertex_tris(faces)
vts

[[0, 1],
 [0, 2, 3],
 [0, 1, 2, 4],
 [1, 2, 5],
 [3],
 [3, 4],
 [4, 5],
 [5, 6],
 [6],
 [6]]

In [19]:
max([len(x) for  x in vts])

4

In [40]:
r, c = len(vts), max([len(x) for  x in vts])
tri_indices = torch.zeros(r, c, dtype=torch.long)
tri_weights = torch.zeros(r, c, dtype=torch.long)

def vertex_tri_maps(faces):
    vts = vertex_tris(faces)
    r, c = len(vts), max([len(x) for  x in vts])
    res = {
        "indices": torch.zeros(r, c, dtype=torch.long), 
        "weights": torch.zeros(r, c), 
    }
    for r, tris in enumerate(vts):
        weight = 1. / len(tris)
        for c, tri_id in enumerate(tris):
            res['indices'][r, c] = tri_id
            res['weights'][r, c] = weight
    return res
vert_tri_map =  vertex_tri_maps(ico_faces)
vert_tri_map

{'indices': tensor([[ 0,  1,  2,  3,  4],
         [ 1,  2,  5,  9, 19],
         [ 7, 11, 12, 16, 17],
         [10, 11, 12, 13, 14],
         [ 6, 10, 11, 15, 16],
         [ 0,  1,  5,  6, 15],
         [ 8, 12, 13, 17, 18],
         [ 2,  3,  8,  9, 18],
         [ 9, 13, 14, 18, 19],
         [ 5, 10, 14, 15, 19],
         [ 3,  4,  7,  8, 17],
         [ 0,  4,  6,  7, 16]]),
 'weights': tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0

In [41]:
face_norm[vert_tri_map['indices']].shape

torch.Size([12, 5, 3])

In [42]:
vert_tri_map['weights'].shape

torch.Size([12, 5])

In [43]:
vert_tri_map['weights'].unsqueeze(dim=-1).shape

torch.Size([12, 5, 1])

In [44]:
vert_tri_weights =  vert_tri_map['weights'].unsqueeze(dim=-1).to(device)
weighted_face_norm = face_norm[vert_tri_map['indices']] * vert_tri_weights
weighted_face_norm.shape

torch.Size([12, 5, 3])

In [50]:
vertex_normals = weighted_face_norm.sum(dim=-2)
vertex_normals = F.normalize(vertex_normals, p=2, dim=-1)
print(vertex_normals.shape)
vertex_normals

torch.Size([12, 3])


tensor([[    -0.5257,      0.8507,     -0.0000],
        [     0.5257,      0.8507,      0.0000],
        [    -0.5257,     -0.8507,      0.0000],
        [     0.5257,     -0.8507,     -0.0000],
        [     0.0000,     -0.5257,      0.8507],
        [    -0.0000,      0.5257,      0.8507],
        [     0.0000,     -0.5257,     -0.8507],
        [     0.0000,      0.5257,     -0.8507],
        [     0.8507,      0.0000,     -0.5257],
        [     0.8507,      0.0000,      0.5257],
        [    -0.8507,     -0.0000,     -0.5257],
        [    -0.8507,      0.0000,      0.5257]], device='cuda:0')

In [46]:
torch.allclose(ico_vertex_normals, vertex_normals)

False

In [47]:
ico_vertex_normals

tensor([[    -0.5257,      0.8507,     -0.0000],
        [     0.5257,      0.8507,      0.0000],
        [    -0.5257,     -0.8507,      0.0000],
        [     0.5257,     -0.8507,     -0.0000],
        [    -0.0000,     -0.5257,      0.8507],
        [     0.0000,      0.5257,      0.8507],
        [     0.0000,     -0.5257,     -0.8507],
        [    -0.0000,      0.5257,     -0.8507],
        [     0.8507,     -0.0000,     -0.5257],
        [     0.8507,      0.0000,      0.5257],
        [    -0.8507,      0.0000,     -0.5257],
        [    -0.8507,     -0.0000,      0.5257]], device='cuda:0')

In [48]:
 F.normalize(vertex_normals, p=2, dim=-1)

tensor([[    -0.5257,      0.8507,     -0.0000],
        [     0.5257,      0.8507,      0.0000],
        [    -0.5257,     -0.8507,      0.0000],
        [     0.5257,     -0.8507,     -0.0000],
        [     0.0000,     -0.5257,      0.8507],
        [    -0.0000,      0.5257,      0.8507],
        [     0.0000,     -0.5257,     -0.8507],
        [     0.0000,      0.5257,     -0.8507],
        [     0.8507,      0.0000,     -0.5257],
        [     0.8507,      0.0000,      0.5257],
        [    -0.8507,     -0.0000,     -0.5257],
        [    -0.8507,      0.0000,      0.5257]], device='cuda:0')

In [51]:
torch.allclose(ico_vertex_normals, vertex_normals, )

False

In [56]:
(ico_vertex_normals - vertex_normals).max().item()

5.7220458984375e-06